In [ ]:
import numpy as np
import tensorflow as tf
import lime
import shap
import matplotlib.pyplot as plt
from lime import lime_image
from tensorflow.keras.models import load_model
from skimage.segmentation import mark_boundaries

# Step 1: Load the model
model_path = "C:\Users\Ahmed Raza\Downloads\MRI-Brain-Tumor-Detection-main\MRI-Brain-Tumor-Detection-main\models\brain_tumor_cnn_classifier.keras"  # replace with your model path
model = load_model(model_path)

# Step 2: Define the prediction function
def predict_fn(images):
    return model.predict(images)

# Step 3: Load or define a sample image
# Assuming images are preprocessed as required by the model
sample_image = np.expand_dims(preprocessed_image, axis=0)  # Preprocess the image as per your model's requirement

# Step 4: LIME Explanation
explainer_lime = lime_image.LimeImageExplainer()
explanation = explainer_lime.explain_instance(
    image=preprocessed_image,
    classifier_fn=predict_fn,
    top_labels=3,
    hide_color=0,
    num_samples=1000
)

# Display LIME explanation for the top label
temp, mask = explanation.get_image_and_mask(
    label=explanation.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=False
)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.title("LIME Explanation")
plt.axis("off")
plt.show()

# Step 5: SHAP Explanation
background = np.random.rand(50, *sample_image.shape[1:])  # Provide a suitable background for SHAP explainer
explainer_shap = shap.DeepExplainer(model, background)
shap_values = explainer_shap.shap_values(sample_image)

# Display SHAP explanation for each class
for i, shap_value in enumerate(shap_values):
    shap.image_plot(shap_value, sample_image, show=True)
    plt.title(f"SHAP Explanation for Class {i}")
    plt.axis("off")
